<a href="https://colab.research.google.com/github/knuffelbeer/leren_en_beslissen_public/blob/main/company_emissions_preprocessing_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
# Import packages
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
from sklearn.neural_network import MLPRegressor

# Import modules

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

#open dataset
df0 = pd.read_csv("20220105_company_emissions_data.csv", sep=',')

#drop irrelevant columns from dataset
df1 = df0.drop(columns=['WorkforceDataPoints.AsianMinoritiesEmployeesPercent',
                       'WorkforceDataPoints.AsianMinoritiesManagersPercent', 
                      'WorkforceDataPoints.AverageEmployeeLengthOfService',
                      'WorkforceDataPoints.AverageTrainingHours', 
                      'WorkforceDataPoints.BbbeeLevel', 
                      'WorkforceDataPoints.BlackOrAfricanAmericanMinoritiesEmployeesPercent',
                      'WorkforceDataPoints.BlackOrAfricanAmericanMinoritiesManagersPercent',
                      'WorkforceDataPoints.ContractorAccidents',
                      'WorkforceDataPoints.ContractorFatalities',
                      'WorkforceDataPoints.ContractorLostWorkingDays',
                      'WorkforceDataPoints.DayCareServices',
                      'WorkforceDataPoints.DiversityAndOpportunityControversies',
                      'WorkforceDataPoints.EmployeeAccidents',
                      'WorkforceDataPoints.EmployeeFatalities',
                      'WorkforceDataPoints.EmployeeHealthAndSafetyTrainingHours',
                      'WorkforceDataPoints.EmployeeLostWorkingDays',
                      'WorkforceDataPoints.EmployeeResourceGroups',
                      'WorkforceDataPoints.EmployeeSatisfaction',
                      'WorkforceDataPoints.EmployeesHealthAndSafetyControversies',
                      'WorkforceDataPoints.EmployeesHealthAndSafetyOhsas18001',
                      'WorkforceDataPoints.EmployeesHealthAndSafetyTeam',
                      'WorkforceDataPoints.EmployeesWithDisabilities',
                      'WorkforceDataPoints.FlexibleWorkingHours',
                      'WorkforceDataPoints.GenderPayGapPercentage',
                      'WorkforceDataPoints.HealthAndSafetyTraining',
                      'WorkforceDataPoints.HispanicOrLatinoMinoritiesEmployeesPercent',
                      'WorkforceDataPoints.HispanicOrLatinoMinoritiesManagersPercent',
                      'WorkforceDataPoints.HivAidsProgram',
                      'WorkforceDataPoints.HrcCorporateEqualityIndex',
                      'WorkforceDataPoints.HsmsCertifiedPercentage',
                      'WorkforceDataPoints.InternalPromotion',
                      'WorkforceDataPoints.InvoluntaryTurnoverOfEmployees',
                      'WorkforceDataPoints.LostTimeInjuryRateContractors',
                      'WorkforceDataPoints.LostTimeInjuryRateEmployees',
                      'WorkforceDataPoints.LostTimeInjuryRateTotal',
                      'WorkforceDataPoints.LostWorkingDays',
                      'WorkforceDataPoints.ManagementDepartures',
                      'WorkforceDataPoints.ManagementTraining',
                      'WorkforceDataPoints.MinoritiesEmployeesPercent',
                      'WorkforceDataPoints.MinoritiesManagersPercent',
                      'WorkforceDataPoints.MinoritiesSalaryGapPercent',
                      'WorkforceDataPoints.NewWomenEmployees',
                      'WorkforceDataPoints.NumberOfEmployeesFromCsrReporting',
                      'WorkforceDataPoints.OccupationalDiseases',
                      'WorkforceDataPoints.OtherMinoritiesEmployeesPercent',
                      'WorkforceDataPoints.OtherMinoritiesManagersPercent',
                      'WorkforceDataPoints.PolicyCareerDevelopment',
                      'WorkforceDataPoints.PolicyDiversityAndOpportunity',
                      'WorkforceDataPoints.PolicyEmployeeHealthAndSafety',
                      'WorkforceDataPoints.PolicySkillsTraining',
                      'WorkforceDataPoints.PolicySupplyChainHealthAndSafety',
                      'WorkforceDataPoints.RecentDiversityOpportunityControversies',
                      'WorkforceDataPoints.RecentEmployeeHealthAndSafetyControversies',
                      'WorkforceDataPoints.RecentWagesWorkingConditionControversies',
                      'WorkforceDataPoints.SalariesAndWagesFromCsrReporting',
                      'WorkforceDataPoints.Strikes',
                      'WorkforceDataPoints.SupplierESGTraining',
                      'WorkforceDataPoints.SupplyChainHealthAndSafetyImprovements',
                      'WorkforceDataPoints.SupplyChainHealthAndSafetyTraining',
                      'WorkforceDataPoints.TargetsDiversityAndOpportunity',
                      'WorkforceDataPoints.TotalInjuryRateContractors',
                      'WorkforceDataPoints.TotalInjuryRateEmployees',
                      'WorkforceDataPoints.TotalInjuryRateTotal',
                      'WorkforceDataPoints.TradeUnionRepresentation',
                      'WorkforceDataPoints.TrainingCostsTotal',
                      'WorkforceDataPoints.TrainingHoursTotal',
                      'WorkforceDataPoints.TurnoverOfEmployees',
                      'WorkforceDataPoints.VoluntaryTurnoverOfEmployees',
                      'WorkforceDataPoints.WagesWorkingConditionControversiesCount',
                      'WorkforceDataPoints.WhiteMinoritiesEmployeesPercent',
                      'WorkforceDataPoints.WhiteMinoritiesManagersPercent',
                      'WorkforceDataPoints.WomenEmployees',
                      'WorkforceDataPoints.WomenManagers',
                      'WorkforceIndicators.AnnouncedLayOffsToTotalEmployees',
                      'WorkforceIndicators.HealthAndSafetyPolicy',
                      'WorkforceIndicators.InjuriesToMillionHours',
                      'WorkforceIndicators.LostDaysToTotalDays',
                      'WorkforceIndicators.NetEmploymentCreation',
                      'WorkforceIndicators.SalaryGap',
                      'WorkforceIndicators.TrainingAndDevelopmentPolicy',
                      'WorkforceIndicators.TrainingCostsPerEmployee',
                      'WorkforceIndicators.WagesWorkingConditionControversies'],
                      axis=1)
    
#number of columns in (cleaned) dataset
len(df0.columns)
len(df1.columns)

df1.sort_values(['GTAP sector'], ascending=[True], inplace=True)

#save cleaned dataset as new file
df = df1.to_csv('company_emissions_cleaned.csv', index=True)

In [24]:
#drop rows with no data in either the year or target collumn
df2 = df1.dropna(subset=['EmissionDataPoints.CO2EquivalentsEmissionDirectScope1', 'StatementDetails.FinancialPeriodFiscalYear'])

In [25]:
df2_names_clean = df2.rename(columns={"GTAP sector": "sector", "EmissionDataPoints.CO2EquivalentsEmissionDirectScope1" :
                                     "target", "StatementDetails.FinancialPeriodFiscalYear" : "year"})

# replace sector with int for use in neural network and remove rows with null sector
df_sector_int = df2_names_clean
for i in range(len(df2_names_clean['sector'].value_counts())):
    df_sector_int = df_sector_int.replace(df2_names_clean['sector'].value_counts().index[i], i + 1)

df_sector_int = df_sector_int[df_sector_int.sector != 1]


#creates small datasets for testing
df = df_sector_int[["year", "target", "sector"]]

# Dit doet het niet echt
#df_small.describe().transpose()
#target_column = ['target'] 
#features = list(set(list(df_small.columns))-set(target_column))
#df_small[features] = df_small[features]/df_small[features].max()
#df_small.describe().transpose()

#splits data into training test data by year 
training_data = df[df.year != 2020]
test_data = df[df.year == 2020]


#link naar nuttige beegin stappen: https://www.pluralsight.com/guides/machine-learning-neural-networks-scikit-learn
#link naar sckitlearn : https://scikit-learn.org/stable/modules/neural_networks_supervised.html



In [26]:
target_column = ['target'] 
predictors = list(set(list(df.columns))-set(target_column))
df[predictors] = df[predictors]/df[predictors].max()
df.describe().transpose()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,count,mean,std,min,25%,50%,75%,max
year,20846.0,9.971354e-01,1.979149e-03,0.991089,0.995545,0.997525,0.999010,1.000000e+00
target,20846.0,3.771560e+06,2.949993e+07,0.000000,7149.500000,55537.620000,607012.750000,3.619808e+09
sector,20846.0,2.497186e-01,2.053802e-01,0.038462,0.076923,0.192308,0.365385,1.000000e+00


In [27]:
X = df[predictors].values
y = df[target_column].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=40)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.10, random_state=40)
print(X_train.shape); print(X_test.shape)

(16884, 2)
(2085, 2)


In [28]:
print(y_train.shape)

(16884, 1)


In [29]:
"""In this step, we will build the neural network model using the scikit-learn library's estimator object, 'Multi-Layer Perceptron Classifier'. The first line of code (shown below) imports 'MLPClassifier'.

The second line instantiates the model with the 'hidden_layer_sizes' argument set to three layers, which has the same number of neurons as the count of features in the dataset. We will also select 'relu' as the activation function and 'adam' as the solver for weight optimization. To learn more about 'relu' and 'adam', please refer to the Deep Learning with Keras guides, the links of which are given at the end of this guide.

The third line of code fits the model to the training data, while the fourth and fifth lines use the trained model to generate predictions on the training and test dataset, respectively."""

# FEEDBACK, for regression probelem, use regressor, not classifier
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(hidden_layer_sizes=(8,8,8), activation='relu', solver='adam', max_iter=500)
print(y_train.shape)
mlp.fit(X_train, y_train.ravel())


# FEEDBACK, use train, val, test, not just train test
mlp.predict(X_test)

(16884, 1)


array([3544624.94465286, 3766834.33471231, 3687517.05301283, ...,
       3782576.78975129, 4008227.75066201, 3602177.02232363])

In [30]:
# FEEDBACK, for regression problem, use regression metrics e.g. Mean squared error
mlp.score(X_test, y_test)

0.0003161075032513949

In [31]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators = 500, random_state = 0)
rfr.fit(X_train, y_train.ravel())
rfr.predict(X_test)

array([1.40208501e+04, 2.14106111e+05, 3.60758399e+05, ...,
       4.06737961e+05, 1.94041722e+05, 2.68777892e+07])

In [32]:
rfr.score(X_test, y_test)

-6.411681184700138